<a href="https://colab.research.google.com/github/Crapho/Crapho/blob/main/IMD_data_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install geopandas
!pip install imdlib
!pip install shapefile

!pip install seaborne
!pip install polygon
!pip install pygeos

import geopandas
import pandas as pd
import numpy as np
import imdlib as imd
import netCDF4 as nc
import shapefile as shp
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Polygon
import pygeos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement shapefile (from versions: none)
ERROR: No matching distribution found for shapefile
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement seaborne (from versions: none)
ERROR: No matching distribution found for seaborne
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
!pip install pyshp
import pyshp 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ModuleNotFoundError: ignored

In [16]:
start_yr = 2019
end_yr = 2020
variable = 'rain'

data = imd.get_data(variable,start_yr, end_yr, fn_format='yearwise')
ds = data.get_xarray()

Downloading: rain for year 2019
Downloading: rain for year 2020
Download Successful !!!


In [17]:
ds1 = ds.to_dataframe()

In [19]:
ds2 = ds1.reset_index()

In [20]:
ds2 = geopandas.GeoDataFrame(
    ds2, geometry=geopandas.points_from_xy(ds2.lon, ds2.lat))

In [21]:
ds2 = ds2[ds2.rain != -999]
print(ds2)

               time    lat    lon  rain                   geometry
987      2019-01-01   8.25  77.00   0.0   POINT (77.00000 8.25000)
988      2019-01-01   8.25  77.25   0.0   POINT (77.25000 8.25000)
989      2019-01-01   8.25  77.50   0.0   POINT (77.50000 8.25000)
990      2019-01-01   8.25  77.75   0.0   POINT (77.75000 8.25000)
1121     2019-01-01   8.50  76.75   0.0   POINT (76.75000 8.50000)
...             ...    ...    ...   ...                        ...
12729456 2020-12-31  37.00  75.50   0.0  POINT (75.50000 37.00000)
12729587 2020-12-31  37.25  74.50   0.0  POINT (74.50000 37.25000)
12729588 2020-12-31  37.25  74.75   0.0  POINT (74.75000 37.25000)
12729589 2020-12-31  37.25  75.00   0.0  POINT (75.00000 37.25000)
12729590 2020-12-31  37.25  75.25   0.0  POINT (75.25000 37.25000)

[3628684 rows x 5 columns]


In [22]:
sns.set(style="whitegrid", palette="pastel", color_codes=True)
sns.mpl.rc("figure", figsize=(10,6))


In [ ]:
shp_path = '/content/drive/MyDrive/archive/output.shp'

In [ ]:
sf = shp.Reader(shp_path)

In [ ]:
len(sf.shapes())
sf.records()

In [ ]:
def read_shapefile(sf):
    """
    Read a shapefile into a Pandas dataframe with a 'coords' 
    column holding the geometry information. This uses the pyshp
    package
    """
    fields = [x[0] for x in sf.fields][1:]
    records = sf.records()
    shps = [s.points for s in sf.shapes()]
    df = pd.DataFrame(columns=fields, data=records)
    df = df.assign(coords=shps)
    return df

df = read_shapefile(sf)
df.shape


(693, 14)

In [ ]:
df_geom = []
for i in range(len(df)): 
    df1 = geopandas.GeoSeries(Polygon(df.coords[i]))
    df_geom.append(df1)
df_geom1 = pd.DataFrame(df_geom)
df = df.join(df_geom1).rename(columns={0: "geometry"}, errors="raise")

In [ ]:
df1 = geopandas.GeoDataFrame(
    df, geometry=df.geometry)


In [ ]:
ds_merged = geopandas.sjoin_nearest(df1,ds2, how="inner", max_distance=50)
ds_merged[ds_merged.rain != -999]

,objectid,statecode,statename,state_ut,distcode,distname,distarea,totalpopul,totalhh,totpopmale,totpopfema,st_areasha,st_lengths,coords,geometry,index_right,time,lat,lon,rain
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"[(79.52658999977145, 29.05542999993833), (79.5...","POLYGON ((79.52659 29.05543, 79.52550 29.05545...",16748285,2020-08-19,29.5,79.00,21.753336
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"[(79.52658999977145, 29.05542999993833), (79.5...","POLYGON ((79.52659 29.05543, 79.52550 29.05545...",17218490,2020-09-15,29.5,79.00,0.000000
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"[(79.52658999977145, 29.05542999993833), (79.5...","POLYGON ((79.52659 29.05543, 79.52550 29.05545...",10722695,2019-09-08,29.5,79.00,0.479617
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"[(79.52658999977145, 29.05542999993833), (79.5...","POLYGON ((79.52659 29.05543, 79.52550 29.05545...",5724590,2018-11-25,29.5,79.00,0.000000
0,1,05,Uttarakhand,STATE,066,Nainital,4251.0,954605.0,191383.0,493666.0,460939.0,5.322546e+09,5.061827e+05,"[(79.52658999977145, 29.05542999993833), (79.5...","POLYGON ((79.52659 29.05543, 79.52550 29.05545...",14501750,2020-04-12,29.5,79.00,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692,34,01,Ladakh,STATE,003,Leh (Ladakh),45110.0,133487.0,21909.0,78971.0,54516.0,1.181149e+11,2.283246e+06,"[(80.06766999996529, 34.78624000024062), (80.0...","POLYGON ((80.06767 34.78624, 80.06849 34.77767...",13267169,2020-02-01,33.0,77.50,0.000000
692,34,01,Ladakh,STATE,003,Leh (Ladakh),45110.0,133487.0,21909.0,78971.0,54516.0,1.181149e+11,2.283246e+06,"[(80.06766999996529, 34.78624000024062), (80.0...","POLYGON ((80.06767 34.78624, 80.06849 34.77767...",17411939,2020-09-26,33.0,77.50,0.858357
692,34,01,Ladakh,STATE,003,Leh (Ladakh),45110.0,133487.0,21909.0,78971.0,54516.0,1.181149e+11,2.283246e+06,"[(80.06766999996529, 34.78624000024062), (80.0...","POLYGON ((80.06767 34.78624, 80.06849 34.77767...",16436699,2020-08-01,33.0,77.50,3.270911
692,34,01,Ladakh,STATE,003,Leh (Ladakh),45110.0,133487.0,21909.0,78971.0,54516.0,1.181149e+11,2.283246e+06,"[(80.06766999996529, 34.78624000024062), (80.0...","POLYGON ((80.06767 34.78624, 80.06849 34.77767...",4089465,2018-08-23,33.0,77.75,0.634161


In [ ]:
ds_pivot = pd.pivot_table(ds_merged, values = ['rain'], index=['time'], columns = ['statename'], aggfunc = np.mean )
ds_pivot1 = ds_pivot.reset_index()
final1 = pd.melt(ds_pivot1, id_vars = ['time'], value_name='avg_rain')


In [ ]:
final1

,time,NaN,statename,avg_rain
0,2018-01-01,rain,Andaman & Nicobar Islands,0.0
1,2018-01-02,rain,Andaman & Nicobar Islands,0.0
2,2018-01-03,rain,Andaman & Nicobar Islands,0.0
3,2018-01-04,rain,Andaman & Nicobar Islands,0.0
4,2018-01-05,rain,Andaman & Nicobar Islands,0.0
...,...,...,...,...
40547,2020-12-27,rain,West Bengal,0.0
40548,2020-12-28,rain,West Bengal,0.0
40549,2020-12-29,rain,West Bengal,0.0
40550,2020-12-30,rain,West Bengal,0.0
